In [9]:
%cd /Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt

/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt


In [10]:
path = '/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/'
block_h, block_w = (32,32)

load_Case_1_1_Model = 'Model_Case_1_1.h5'
load_Case_1_2_Model = 'Model_Case_1_2.h5'
load_Case_1_3_Model = 'Model_Case_1_3.h5'
load_Case_1_4_Model = 'Model_Case_1_4.h5'
load_Case_1_5_Model = 'Model_Case_1_5.h5'
load_Case_2_1_Model = 'Model_Case_2_1.h5'
load_Case_2_2_Model = 'Model_Case_2_2.h5'
load_Case_2_3_Model = 'Model_Case_2_3.h5'
load_Case_2_4_Model = 'Model_Case_2_4.h5'
load_Case_2_5_Model = 'Model_Case_2_5.h5'
load_Case_3_1_Model = 'Model_Case_3_1.h5'
load_Case_3_2_Model = 'Model_Case_3_2.h5'
load_Case_3_3_Model = 'Model_Case_3_3.h5'
load_Case_3_4_Model = 'Model_Case_3_4.h5'
load_Case_3_5_Model = 'Model_Case_3_5.h5'
load_Case_4_1_Model = 'Model_Case_4_1.h5'
load_Case_4_2_Model = 'Model_Case_4_2.h5'
load_Case_4_3_Model = 'Model_Case_4_3.h5'
load_Case_4_4_Model = 'Model_Case_4_4.h5'
load_Case_4_5_Model = 'Model_Case_4_5.h5'
load_Case_5_1_Model = 'Model_Case_5_1.h5'
load_Case_5_2_Model = 'Model_Case_5_2.h5'
load_Case_5_3_Model = 'Model_Case_5_3.h5'
load_Case_5_4_Model = 'Model_Case_5_4.h5'
load_Case_5_5_Model = 'Model_Case_5_5.h5'


In [11]:
# Function Definition --> For reading the Images and append it to a list

def Reading_Data():
    from tqdm import tqdm  # It is a progress bar, Used to track the estimated remaining time.
    import cv2  # Used for Images processing.
    splitpath = os.listdir(Split_Path)
    if '.DS_Store' in splitpath:
        splitpath.remove('.DS_Store')
    for category in splitpath:
        Category_Path = os.path.join(Split_Path, category + '/')  # Joining path for "MGMT_positive & MGMT_negative".
        class_num = splitpath.index(category)  # Initializing index for each class.
        categorypath = os.listdir(Category_Path)
        if '.DS_Store' in categorypath:
            categorypath.remove('.DS_Store')
        for img_folder in tqdm(categorypath):  # Working with the folders of images
            Img_Path = os.path.join(Category_Path, img_folder + '/')
            imagepath = os.listdir(Img_Path)
            if '.DS_Store' in imagepath:
                imagepath.remove('.DS_Store')
            for img in imagepath:  # listing all images present in the image folder.
                try:
                    img_array = cv2.imread(os.path.join(Img_Path, img), cv2.IMREAD_GRAYSCALE)
                    test_data.append([img_array, class_num])  # Saving images with their corresponding class labels.
                except Exception as e:
                    pass

In [12]:
# Function Defination --> For Initilizing all features & labels of the processed image in the list X & Y

def Initilizing_Features_Labels():
    for features, label in test_data:
        X.append(features)
        y.append(label)

    print('List Size: ', len(X), len(y))

In [13]:
# Function Defination --> Converting List into Numpy Array for faster calculation & it also take less space.

def Converting(block_h, block_w):
    import numpy as np  # Used for array operations.
    global X, y

    X = np.array(X).reshape(-1, block_h, block_w, 1)  # -1 is added to solve dimension mismatch while converting list to an array.
    y = np.array(y)

    # Print shape of the loading data
    print('Shape of X:\t', X.shape)
    print('Shape of y:\t', y.shape)

In [14]:
# Function Definition --> Loading the model

def Load_Model():
    if case == 'Case_1':
        my_model = tf.keras.models.load_model(
            path + 'Outputs/BRATS/t1ce_output/' + load_Case_1_2_Model)
    elif case == 'Case_2':
        my_model = tf.keras.models.load_model(
            path + 'Outputs/BRATS/t1ce_output/' + load_Case_2_2_Model)
    elif case == 'Case_3':
        my_model = tf.keras.models.load_model(
            path + 'Outputs/BRATS/t1ce_output/' + load_Case_3_2_Model)
    elif case == 'Case_4':
        my_model = tf.keras.models.load_model(
            path + 'Outputs/BRATS/t1ce_output/' + load_Case_4_2_Model)
    elif case == 'Case_5':
        my_model = tf.keras.models.load_model(
            path + 'Outputs/BRATS/t1ce_output/' + load_Case_5_2_Model)
            
    return my_model

In [15]:
# Function Definition --> Plotting the AUC-ROC Curve

def Plotting_AUC_ROC_Curve(X, y, model):
    import sklearn.metrics as metrics
    import matplotlib.pyplot as plt

    # calculate the fpr and tpr for all thresholds of the classification
    probs = model.predict(X)
    preds = probs[:,1]
    fpr, tpr, threshold = metrics.roc_curve(y, preds)
    roc_auc = metrics.auc(fpr, tpr)

    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])

    plt.title('Receiver Operating Characteristic')
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')

    plt.show()
    plt.savefig(path + 'Outputs/AUC-ROC_Graph_' + case + '.png')

In [16]:
# All Function Calls
import os
from tqdm import tqdm  # It is a progress bar, Used to track the estimated remaining time.
import tensorflow as tf
from tensorflow.keras.models import load_model
from keras.utils.np_utils import to_categorical
import numpy as np

PATH = path + 'Data/BRATS/mod_data/'
Work_Dir = PATH + 'Working_Data/'
Result_File = path + 'Outputs/BRATS/t1ce_output/Final_Test_Result.txt'
workdir = os.listdir(Work_Dir)
if '.DS_Store' in workdir:
    workdir.remove('.DS_Store')
# Calling Functions:
for case in workdir:
    print(case)
    Case_Path = os.path.join(Work_Dir, case)  # Joining path for "Cases'.
    Split_Path = Case_Path + '/test/'  # Joining "test" folder with "Cases" path.

    # Creating list for storing processed data
    test_data = []

    # Function Call --> For reading the Images and append it to a list
    Reading_Data()

    # Printing Length of the test data.
    print('Length of the total test data: ' + str(len(test_data)))
    
    # Initilizing all features & labels of the processed image in the list X & Y.
    X = []
    y = []

    # Function Call --> For Initilizing all features & labels of the processed image in the list X & Y
    Initilizing_Features_Labels()

    # Function Call --> For Converting List into Numpy Array for faster calculation & it also take less space.
    Converting(block_h, block_w)

    results = []

    # Function call --> Loading the model
    my_model = Load_Model()
    print("my_model")
    
    # Function Call --> Plotting the AUC-ROC Curve
    
    Plotting_AUC_ROC_Curve(X, y, my_model)

    print('\n')

os.chdir(path)
print('Done..!!')

Case_5


NotADirectoryError: [Errno 20] Not a directory: '/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/Data/BRATS/mod_data/Working_Data/Case_5/train/y.pickle/'